# Toy study: Gaussian on a circle

## Setup

In [1]:
import sys
from collections import OrderedDict
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import torch
from torch.utils.data import TensorDataset
import logging

sys.path.append("../../")
from manifold_flow import transforms, utils, distributions, training
from manifold_flow.flows import Flow, ManifoldFlow
from manifold_flow import nn as nn_
from experiments.simulators.spherical_simulator import SphericalGaussianSimulator
from experiments.utils.models import create_vector_transform

%matplotlib inline

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "experiments" not in key and "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)


### Settings

In [2]:
epochs = 120
n_train = 10000
epsilon = 0.01
train = True

### Data

In [3]:
simulator = SphericalGaussianSimulator(latent_dim=1, data_dim=2, epsilon=epsilon)
x_sim = simulator.sample(n_train)
x_sim_tensor = torch.from_numpy(x_sim)
train_dataset = TensorDataset(x_sim_tensor, x_sim_tensor)

In [4]:
# res = 200
# x_range = np.linspace(-1.4,1.4,res)
# y_range = np.linspace(-1.4,1.4,res)
# xx, yy = np.meshgrid(x_range, y_range)
# x_grid = np.concatenate((xx.reshape((-1,1)), yy.reshape((-1,1))), axis=1)
# logp_grid = simulator.log_density(x_grid, precise=True).reshape((res, res))
# logp_grid[~np.isfinite(logp_grid)] = -1000000.

# zmin, zmax = np.max(logp_grid) - 10, np.max(logp_grid)
# fig = plt.figure(figsize=(5,4))
# ax = plt.gca()

# pcm = plt.imshow(
#     np.clip(logp_grid, zmin, zmax),
#     extent=(-1.4,1.4,-1.4,1.4),
#     origin="lower",
#     cmap="viridis", norm=matplotlib.colors.Normalize(zmin, zmax),
#     interpolation='nearest'
# )
# cb = plt.colorbar(pcm, extend="both")
# plt.scatter(x_sim[::50,0], x_sim[::50,1], s=2., c="black", alpha=1.)

# plt.xlabel("$x_1$")
# plt.ylabel("$x_2$")
# cb.set_label("$\log \; p(x)$")
# plt.tight_layout()

# plt.savefig("../figures/simulator_toy.pdf", dpi=500)

# pixel_size = (x_range[1] - x_range[0]) * (y_range[1] - y_range[0])

# print("Integral over density =", np.sum(np.exp(logp_grid) * pixel_size))

## Set up models

In [5]:
labels = OrderedDict()
labels["truth"] = "Truth"
labels["sf"] = "Standard flow"
labels["pie"] = "PIE (manifold)"
labels["pie_full"] = "PIE (all)"
labels["mf"] = "MF"
labels["mlfl"] = "MLF-L"
labels["mlfa"] = "MLF-A"
labels["mlfot"] = "MLF-OT"
labels["mlfae"] = "MLF-AE"


### Standard flow

In [ ]:
sf = Flow(
    data_dim=2,
    transform=create_vector_transform(2, 10, base_transform_type="affine-coupling"),
)

if train:
    trainer = training.trainer.ManifoldFlowTrainer(sf)
    trainer.train(
        train_dataset,
        [training.losses.nll],
        loss_weights=[1.],
        epochs=epochs,
    )
    torch.save(sf.state_dict(), "../data/models/sf_toy.pt")
else:
    sf.load_state_dict(torch.load("../data/models/sf_toy.pt"))

### Manifold flow (with specified manifold)

In [9]:
mf = ManifoldFlow(
    data_dim=2,
    latent_dim=1,
    inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
    outer_transform=transforms.SphericalCoordinates(n=1, r0=1., azimuthal_offset=-0.5*np.pi)
)

if train:
    trainer = training.trainer.ManifoldFlowTrainer(mf)
    trainer.train(
        train_dataset,
        [training.losses.nll],
        loss_weights=[1.],
        epochs=epochs,
        forward_kwargs={"mode":"mf"}
    )
    torch.save(mf.state_dict(), "../data/models/mf_toy.pt")
else:
    mf.load_state_dict(torch.load("../data/models/mf_toy.010.pt"))

16:12 manifold_flow.training.trainer INFO    Training on CPU with single precision
16:12 manifold_flow.training.trainer INFO    Epoch   6: train loss  3.02396 (nll:  3.024)
16:12 manifold_flow.training.trainer INFO               val. loss   2.94437 (nll:  2.944)
16:12 manifold_flow.training.trainer INFO    Epoch  12: train loss  2.16272 (nll:  2.163)
16:12 manifold_flow.training.trainer INFO               val. loss   2.12582 (nll:  2.126)
16:12 manifold_flow.training.trainer INFO    Epoch  18: train loss  1.76621 (nll:  1.766)
16:12 manifold_flow.training.trainer INFO               val. loss   1.75563 (nll:  1.756)
16:12 manifold_flow.training.trainer INFO    Epoch  24: train loss  1.58474 (nll:  1.585)
16:12 manifold_flow.training.trainer INFO               val. loss   1.58025 (nll:  1.580)
16:13 manifold_flow.training.trainer INFO    Epoch  30: train loss  1.47470 (nll:  1.475)
16:13 manifold_flow.training.trainer INFO               val. loss   1.47297 (nll:  1.473)
16:13 manifold_fl

### PIE

In [ ]:
pie = ManifoldFlow(
    data_dim=2,
    latent_dim=1,
    outer_transform=create_vector_transform(2, 5, base_transform_type="affine-coupling"),
    inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
    pie_epsilon=0.01,
)

if train:
    trainer = training.trainer.ManifoldFlowTrainer(pie)
    trainer.train(
        train_dataset,
        [training.losses.nll],
        loss_weights=[1.],
        epochs=epochs,
        forward_kwargs={"mode":"pie"}
    )
    torch.save(pie.state_dict(), "../data/models/pie_toy.pt")
else:
    pie.load_state_dict(torch.load("../data/models/pie_toy.pt"))
    

### MLF-L

In [11]:
# mlfl = ManifoldFlow(
#     data_dim=2,
#     latent_dim=1,
#     inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
#     outer_transform=create_vector_transform(2, 3, base_transform_type="affine-coupling")
# )

# if train:
#     trainer = training.trainer.ManifoldFlowTrainer(mlfl)
#     trainer.train(
#         train_dataset,
#         [training.losses.mse],
#         loss_weights=[100.],
#         epochs=epochs // 3,
#         forward_kwargs={"mode":"projection"}
#     )
#     trainer.train(
#         train_dataset,
#         [training.losses.mse, training.losses.nll],
#         loss_weights=[100., 0.1],
#         epochs=epochs // 3,
#         forward_kwargs={"mode":"mf"}
#     )
#     trainer.train(
#         train_dataset,
#         [training.losses.mse, training.losses.nll],
#         loss_weights=[0., 1.],
#         epochs=epochs // 3,
#         parameters=mf.inner_transform.parameters(),
#         forward_kwargs={"mode":"mf"}
#     )
#     torch.save(mlfl.state_dict(), "../data/models/mlfl_toy.pt")
# else:
#     mlfl.load_state_dict(torch.load("../data/models/mlfl_toy.pt"))

### MLF-A

In [ ]:
mlfa = ManifoldFlow(
    data_dim=2,
    latent_dim=1,
    inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
    outer_transform=create_vector_transform(2, 5, base_transform_type="affine-coupling")
)

if train:
    trainer = training.ManifoldFlowTrainer(mlfa)
    metatrainer = training.AlternatingTrainer(mlfa, trainer, trainer)
    metatrainer.train(
        train_dataset,
        loss_functions=[training.losses.mse, training.losses.nll],
        loss_function_trainers=[0, 1],
        loss_labels=["MSE", "NLL"],
        loss_weights=[100., 0.1],
        epochs=epochs,
        parameters=[mlfa.parameters(), mlfa.inner_transform.parameters()],
        trainer_kwargs=[{"forward_kwargs": {"mode": "projection"}}, {"forward_kwargs": {"mode": "pie"}}],
    )
    torch.save(mlfa.state_dict(), "../data/models/mlfa_toy.pt")
else:
    mlfa.load_state_dict(torch.load("../data/models/mlfa_toy.pt"))

16:28 manifold_flow.training.trainer INFO    Training on CPU with single precision
16:28 manifold_flow.training.trainer INFO    Training on CPU with single precision
16:28 manifold_flow.training.trainer INFO    Epoch   6: train loss  0.56228 (MSE:  0.002, NLL:  3.818)
16:28 manifold_flow.training.trainer INFO               val. loss   0.47213 (MSE:  0.001, NLL:  3.839)
16:28 manifold_flow.training.trainer INFO    Epoch  12: train loss  0.48131 (MSE:  0.002, NLL:  2.680)
16:28 manifold_flow.training.trainer INFO               val. loss   0.32946 (MSE:  0.001, NLL:  2.677)
16:29 manifold_flow.training.trainer INFO    Epoch  18: train loss  0.31495 (MSE:  0.001, NLL:  2.019)
16:29 manifold_flow.training.trainer INFO               val. loss   0.24908 (MSE:  0.000, NLL:  2.068)
16:29 manifold_flow.training.trainer INFO    Epoch  24: train loss  0.22932 (MSE:  0.001, NLL:  1.636)
16:29 manifold_flow.training.trainer INFO               val. loss   0.22231 (MSE:  0.000, NLL:  1.779)
16:30 mani

### MLF-OT

In [ ]:
mlfot = ManifoldFlow(
    data_dim=2,
    latent_dim=1,
    inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
    outer_transform=create_vector_transform(2, 5, base_transform_type="affine-coupling")
)

if train:
    trainer = training.trainer.GenerativeTrainer(mlfot)
    trainer.train(
        train_dataset,
        [training.losses.make_sinkhorn_divergence()],
        loss_weights=[100.],
        epochs=10*epochs,
        batch_size=1000,
    )
    torch.save(mlfot.state_dict(), "../data/models/mlfot_toy.pt")
else:
    mlfot.load_state_dict(torch.load("../data/models/mlfot_toy.pt"))

### MLF-AE

In [ ]:
mlfae = ManifoldFlow(
    data_dim=2,
    latent_dim=1,
    inner_transform=transforms.ConditionalAffineScalarTransform(features=1),
    outer_transform=create_vector_transform(2, 5, base_transform_type="affine-coupling")
)

if train:
    trainer = training.trainer.ManifoldFlowTrainer(mlfae)
    trainer.train(
        train_dataset,
        [training.losses.mse],
        loss_weights=[100.],
        epochs=epochs,
        forward_kwargs={"mode":"projection"}
    )
    torch.save(mlfae.state_dict(), "../data/models/mlfae_toy.pt")
else:
    mlfae.load_state_dict(torch.load("../data/models/mlfae_toy.pt"))

## Evaluate the models

In [ ]:
sf.eval()
mf.eval()
pie.eval()
# mlfl.eval()
mlfa.eval()
mlfot.eval()
mlfae.eval()

### Sample from models

In [ ]:
x_gen = OrderedDict()

x_gen["truth"] = x_sim[:n]
x_gen["sf"] = sf.sample(n=1000).detach().numpy()
x_gen["mf"] = mf.sample(n=1000).detach().numpy()
x_gen["pie"] = pie.sample(n=1000).detach().numpy()
x_gen["pie_full"] = pie.sample(n=1000, sample_orthogonal=True).detach().numpy()
# x_gen["mlfl"] = mlfl.sample(n=1000).detach().numpy()
x_gen["mlfa"] = mlfa.sample(n=1000).detach().numpy()
x_gen["mlfot"] = mlfot.sample(n=1000).detach().numpy()
x_gen["mlfae"] = mlfae.sample(n=1000).detach().numpy()


### Evaluate model likelihood over data space

In [ ]:
res = 200
boundary = 1.5
im_extent = boundary + 0.5 * (2.*boundary/(res - 1))

x_range = np.linspace(-boundary, boundary, res)
y_range = np.linspace(-boundary, boundary, res)
xx, yy = np.meshgrid(x_range, y_range)
x_grid = np.concatenate((xx.reshape((-1,1)), yy.reshape((-1,1))), axis=1)
x_grid_tensor = torch.FloatTensor(x_grid)


logp_grid = OrderedDict()
logp_grid["truth"] = simulator.log_density(x_grid).reshape((res, res))
logp_grid["truth"][~np.isfinite(logp_grid["truth"])] = -1000000.
logp_grid["sf"] = sf.log_prob(x_grid_tensor).detach().numpy().reshape((res, res))
logp_grid["pie_full"] = pie.log_prob(x_grid_tensor, mode="pie").detach().numpy().reshape((res, res))


In [ ]:
def likelihood_along_manifold(model, mode="mf", zmin=-50., zmax=50., n_samples=1000, epsilon=0.02, max_insert=100):
    # Sample
    u = torch.linspace(zmin, zmax, n_samples).view(-1,1)
    x = model.sample(n=n_samples, u=u).detach().numpy()
    u = u.numpy().flatten()
    
    def in_box(x):
        return np.abs(x[0]) < 1.5 or np.abs(x[1]) < 1.5
    
    # Interpolate
    x_interpol = [x[0]]
    for x_prev, x_now, u_prev, u_now  in zip(x[:-1], x[1:], u[:-1], u[1:]):
        
        # Check if we should generate more points in between
        distance = np.linalg.norm(x_now-x_prev)
        if distance > epsilon and (in_box(x_now) or in_box(x_prev)):
            n_insert = min(int(distance / epsilon), max_insert)
            u_insert = torch.linspace(u_prev, u_now, n_insert + 2)[1:-1].view(-1,1)
            x_insert = model.sample(n=n_insert, u=u_insert).detach().numpy()
            for x_ in x_insert:
                if in_box(x_):
                    x_interpol.append(x_)
            
        if in_box(x_now):
            x_interpol.append(x_now)
            
    x_interpol = np.array(x_interpol)

    # Evaluate likelihood
    log_probs = model.log_prob(torch.FloatTensor(x_interpol), mode=mode).detach().numpy()

    # Return
    return x_interpol, log_probs


logp_manifold, x_manifold = OrderedDict(), OrderedDict()
x_manifold["mf"], logp_manifold["mf"] = likelihood_along_manifold(mf)
# x_manifold["mlfl"], logp_manifold["mlfl"] = likelihood_along_manifold(mlfl)
x_manifold["mlfa"], logp_manifold["mlfa"] = likelihood_along_manifold(mlfa)
x_manifold["mlfot"], logp_manifold["mlfot"] = likelihood_along_manifold(mlfot)
x_manifold["mlfae"], logp_manifold["mlfae"] = likelihood_along_manifold(mlfae)


### Calculate metrics for generated samples: mean distance from manifold, true likelihood

In [ ]:
def generative_metrics(x, logp_min=-100., d_max=1., summary_fn=np.mean):
    logp_gen = simulator.log_density(x)
    logp_gen[(~np.isfinite(logp_gen)) + (logp_gen<logp_min)] = logp_min
    logp_summary = summary_fn(logp_gen)
    
    d_gen = np.abs(np.sum(x**2, axis=1)**0.5 - 1)
    d_gen[(~np.isfinite(d_gen)) + (d_gen>d_max)] = d_max
    d_summary = summary_fn(d_gen)
    
    return logp_summary, d_summary


logp_gen, d_gen = OrderedDict(), OrderedDict()

for key, val in x_gen.items():
    logp_gen[key], d_gen[key] = generative_metrics(x_gen[key])
    

print("Mean true log likelihood of samples generated from flows (higher is better):")
for key, val in logp_gen.items():
    print("  {:>10.10s}: {:>6.1f}".format(key, val))
    
print("Mean Euclidean distance between samples generated from flows and true manifold (lower is better):")
for key, val in d_gen.items():
    print("  {:>10.10s}: {:>6.4f}".format(key, val))
    

### Visualize generated samples

In [ ]:
n = 500
show = ["truth", "sf", "pie_full", "pie", "mf", "mlfa", "mlfot", "mlfae"]

fig = plt.figure(figsize=(4.*4,4.*2))

for i, key in enumerate(show):
    ax = plt.subplot(2,4,i+1)
    
    plt.scatter(x_gen[key][:n,0], x_gen[key][:n,1], s=10., c="C3", label=labels[key])
    plt.legend()

    plt.xlim(-boundary, boundary)
    plt.ylim(-boundary, boundary)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")

plt.tight_layout()
plt.savefig("../figures/generated_samples_toy.pdf")


### Plot likelihoods

In [ ]:
show = ["truth", "sf", "pie_full", "mf", "mlfa", "mlfot", "mlfae"]

fig = plt.figure(figsize=(5.*5,4.*2))

for i, key in enumerate(show):
    ax = plt.subplot(2,4,i+1)
    
    try:
        x = x_manifold[key]
        logp = logp_manifold[key]
        zmin, zmax = np.max(logp) - 5., np.max(logp)
        
        segments = np.concatenate([x[:-1,np.newaxis,:], x[1:,np.newaxis,:]], axis=1)
        lc = LineCollection(segments, cmap='viridis', norm=plt.Normalize(zmin, zmax))
        lc.set_array(np.clip(logp, zmin, zmax))
        lc.set_linewidth(1.5)
        im = ax.add_collection(lc)

    except KeyError:
        logp = logp_grid[key]
        zmin, zmax = np.max(logp) - 5., np.max(logp)
        im = plt.imshow(
            np.clip(logp, zmin, zmax),
            extent=(-im_extent, im_extent, -im_extent, im_extent),
            origin="lower",
            cmap="viridis",
            norm=matplotlib.colors.Normalize(zmin, zmax),
            interpolation='nearest'
        )
    cb = plt.colorbar(im, extend="both")

    plt.xlim(-im_extent, im_extent)
    plt.ylim(-im_extent, im_extent)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    cb.set_label(r"{} log likelihood".format(labels[key]))
    
plt.tight_layout()
plt.savefig("../figures/log_likelihood_toy.pdf", dpi=600)


## Other plots (not updated to new conventions yet)

### Plot reconstruction performance

In [ ]:
# x_in = SphericalGaussianSimulator(1,2,epsilon=10*epsilon).sample(100)
# x_in = torch.FloatTensor(x_in)
# x_out = mf(x_in)[0]
# x_in, x_out = x_in.detach().numpy(), x_out.detach().numpy()
# dx = x_out - x_in

In [ ]:
# fig = plt.figure(figsize=(5,5))
# ax = plt.gca()
# plt.scatter(x_in[:,0], x_in[:,1], s=15., c="C1")
# plt.scatter(x_out[:,0], x_out[:,1], s=15., c="C0")
# plt.quiver(
#     x_in[:,0], x_in[:,1], dx[:,0], dx[:,1],
#     angles='xy', scale_units='xy', scale=1., width=2.e-3, alpha=1.
# )

# plt.xlim(-1.5,1.5)
# plt.ylim(-1.5,1.5)
# plt.xlabel("$x_1$")
# plt.ylabel("$x_2$")
# plt.tight_layout()

# plt.savefig("../figures/spherical_gaussian_2d_mf_reco.pdf")


### Model anatomy

In [ ]:
# def make_anatomy_plot(model, name, res=25):
#     x_range = np.linspace(-1.5,1.5,res)
#     y_range = np.linspace(-1.5,1.5,res)
#     xx, yy = np.meshgrid(x_range, y_range)
#     x = np.concatenate((xx.reshape((-1,1)), yy.reshape((-1,1))), axis=1)
#     x = torch.FloatTensor(x)
#     x.requires_grad = True

#     # Encode
#     u, h_manifold, h_orthogonal, log_det_outer, log_det_inner = model._encode(x)

#     # Decode
#     x_reco, _, _, inv_jacobian_outer = model._decode(u, mode="mf")
#     _, inv_log_det_inner, inv_log_det_outer, _ = model._decode(u, mode="slice")

#     # inv_jacobian_outer is dx / du, but still need to restrict this to the manifold latents
#     inv_jacobian_outer = inv_jacobian_outer[:, :, : model.latent_dim]
#     # And finally calculate log det (J^T J)
#     jtj = torch.bmm(torch.transpose(inv_jacobian_outer, -2, -1), inv_jacobian_outer)
#     mf_log_det_outer = - 0.5 * torch.slogdet(jtj)[1]

#     # Base log prob
#     log_prob_latent = model.manifold_latent_distribution._log_prob(u, context=None)
    
#     # Plot
#     fig = plt.figure(figsize=(12,9))

#     for panel, (label, quantity, diverging) in enumerate(zip(
#         ["Perp latent", "Manifold latent after outer flow", "Manifold latent after inner flow",
#          "log det outer (PIE)", "log det inner", "Base log prob",
#          "log det outer (MF)", "log det outer (Slice of PIE)", "delta log det outer (MF - Slice)"],
#         [h_orthogonal, h_manifold, u,
#          log_det_outer, - log_det_inner, log_prob_latent,
#          mf_log_det_outer, - inv_log_det_outer, mf_log_det_outer + inv_log_det_outer],
#         [True, True, True, False, False, False, False, False, False]
#     )):
#         ax = plt.subplot(3,3,panel+1)

#         quantity_ = quantity.detach().numpy()
#         quantity_ = quantity_.flatten() + np.zeros((res**2))

#         if diverging:
#             zmin, zmax = - 2. * np.std(quantity_), 2. * np.std(quantity_)
#         else:
#             zmin, zmax = np.mean(quantity_) - 1.5 * np.std(quantity_), np.mean(quantity_) + 1.5 * np.std(quantity_)

#         pcm = plt.imshow(
#             np.clip(quantity_, zmin, zmax).reshape(res, res),
#             extent=(-1.5, 1.5, -1.5, 1.5),
#             origin="lower",
#             cmap="PRGn" if diverging else "viridis",
#             norm=matplotlib.colors.Normalize(zmin, zmax),
#             interpolation='nearest'
#         )
#         cb = plt.colorbar(pcm, extend="both")
#         # plt.scatter(x_gen_mf[::10,0], x_gen_mf[::10,1], s=3., c="black")

#         plt.xlim(-1.5,1.5)
#         plt.ylim(-1.5,1.5)
#         plt.xlabel("$x_1$")
#         plt.ylabel("$x_2$")
#         cb.set_label(label)
#         plt.tight_layout()

#     plt.savefig("../figures/spherical_gaussian_2d_{}_anatomy.pdf".format(name))